# Risk Calculation Examples (v2)
This notebook demonstrates healthcare-specific risk calculations with temporal and equity weights.

## Imports & Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys, json

sys.path.append(str(Path('..').resolve()))
from src import risk_models as rm

df = pd.read_csv('../data/example_inputs.csv')
df.head()

## Parameter Panel (interactive)

In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display
    row = df.iloc[0].to_dict()

    sliders = {
        'probability_failure': widgets.FloatSlider(value=row['probability_failure'], min=0, max=1, step=0.01, description='p_fail'),
        'impact_severity': widgets.FloatSlider(value=row['impact_severity'], min=0, max=1, step=0.01, description='severity'),
        'data_volume': widgets.IntSlider(value=int(row['data_volume']), min=0, max=50000, step=100, description='data_vol'),
        'standardization_maturity': widgets.FloatSlider(value=row['standardization_maturity'], min=0.01, max=1, step=0.01, description='std_maturity'),
        'missed_benefits_usd': widgets.IntSlider(value=int(row['missed_benefits_usd']), min=0, max=5000000, step=10000, description='missed_usd'),
        'time_delay_months': widgets.IntSlider(value=int(row['time_delay_months']), min=0, max=24, step=1, description='delay_mo'),
        'patient_population': widgets.IntSlider(value=int(row['patient_population']), min=1, max=200000, step=100, description='population'),
        'current_performance': widgets.FloatSlider(value=row['current_performance'], min=0, max=1, step=0.01, description='baseline'),
        'system_conflicts': widgets.FloatSlider(value=row['system_conflicts'], min=0, max=1, step=0.01, description='sys_conf'),
        'workflow_disruption': widgets.FloatSlider(value=row['workflow_disruption'], min=0, max=1, step=0.01, description='workflow'),
        'user_confusion': widgets.FloatSlider(value=row['user_confusion'], min=0, max=1, step=0.01, description='confusion'),
        'integration_quality': widgets.FloatSlider(value=row['integration_quality'], min=0, max=1, step=0.01, description='integration'),
        'vulnerability_score': widgets.FloatSlider(value=row['vulnerability_score'], min=0, max=1, step=0.01, description='vulnerab'),
        'access_barriers': widgets.FloatSlider(value=row['access_barriers'], min=0, max=1, step=0.01, description='barriers'),
        'outcome_disparities': widgets.FloatSlider(value=row['outcome_disparities'], min=0, max=1, step=0.01, description='disparity'),
        'discount_rate': widgets.FloatSlider(value=row['discount_rate'], min=0, max=0.2, step=0.005, description='discount'),
        'time_horizon_years': widgets.FloatSlider(value=row['time_horizon_years'], min=0, max=5, step=0.1, description='horizon'),
        'urgency_factor': widgets.FloatSlider(value=row['urgency_factor'], min=0, max=2, step=0.1, description='urgency'),
        'w_impl': widgets.FloatSlider(value=row['w_impl'], min=0, max=1, step=0.05, description='w_impl'),
        'w_opp': widgets.FloatSlider(value=row['w_opp'], min=0, max=1, step=0.05, description='w_opp'),
        'w_int': widgets.FloatSlider(value=row['w_int'], min=0, max=1, step=0.05, description='w_int'),
        'opp_cap_usd_per_patient': widgets.IntSlider(value=int(row['opp_cap_usd_per_patient']), min=1000, max=50000, step=500, description='opp_cap'),
    }

    def on_change(change=None):
        params = {k:(v.value) for k,v in sliders.items()}
        out = rm.total_risk(params)
        print(out)

    box = widgets.VBox(list(sliders.values()))
    display(box)
    on_change()
except Exception as e:
    print("ipywidgets not available; skipping interactive panel.", e)

## Compute per-scenario results

In [ ]:
def compute_row(row):
    params = row.to_dict()
    out = rm.total_risk(params)
    return pd.Series(out)

results = df.apply(compute_row, axis=1)
display(pd.concat([df[['scenario','clinical_area']], results], axis=1))

## Hand-Checkable Validation Example

In [ ]:
val = pd.read_csv('../data/validation_scenarios.csv').iloc[0].to_dict()
out = rm.total_risk(val)
print('Validation output:', out)
print('Manual check embedded in tests/test_suite.py comments.')

## Sensitivity Analysis (Tornado Plot)

In [ ]:
row = df.iloc[0].to_dict()
base = rm.total_risk(row)['r_total']

def vary(param, scale=0.2):
    r = row.copy()
    if isinstance(r[param], (int,float)):
        delta = r[param]*scale if r[param]!=0 else scale
        up = r.copy(); up[param] = r[param] + delta
        dn = r.copy(); dn[param] = max(0, r[param] - delta)
        up_val = rm.total_risk(up)['r_total']
        dn_val = rm.total_risk(dn)['r_total']
        return param, base-dn_val, up_val-base
    return None

params_to_test = ['probability_failure','impact_severity','data_volume','standardization_maturity','missed_benefits_usd','time_delay_months','patient_population','current_performance','system_conflicts','workflow_disruption','user_confusion','integration_quality','vulnerability_score','access_barriers','outcome_disparities','discount_rate','time_horizon_years','urgency_factor']
bars = [vary(p) for p in params_to_test]
bars = [b for b in bars if b]

labels = [b[0] for b in bars]
left = [b[1] for b in bars]
right = [b[2] for b in bars]

y = np.arange(len(labels))
plt.figure(figsize=(8, max(6,len(labels)*0.3)))
for i in range(len(labels)):
    plt.barh(i, right[i])
    plt.barh(i, -left[i])
plt.yticks(y, labels)
plt.axvline(0, linewidth=1)
plt.title('Tornado Analysis (±20%)')
plt.tight_layout()
plt.show()

## Clinical Interpretation

Results should be interpreted with clinical stakeholders. High `opportunity_risk` suggests time-to-benefit losses; high `interaction_risk` flags socio-technical issues needing integration work; equity weights >1 indicate at-risk populations warranting priority.